In [1]:
import numpy as np

In [2]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from lightning.pytorch import Trainer

/home/demancum/miniconda3/envs/cocola/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'  

In [5]:
from contrastive_model.contrastive_model import CoCola
from data.coco_chorales_contrastive_preprocessed import CocoChoralesContrastivePreprocessed
from data.moisesdb_contrastive_preprocessed import MoisesdbContrastivePreprocessed
from data.slakh2100_contrastive_preprocessed import Slakh2100ContrastivePreprocessed
from data.musdb_contrastive_preprocessed import MusdbContrastivePreprocessed
from contrastive_model import constants


In [6]:
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/k5llcdm8/checkpoints/epoch=21-step=21406.ckpt' #bumbling-breeze-34 bs=512 chunk_dur=2s
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/r71kxj7r/checkpoints/epoch=36-step=29119.ckpt' #good-wave-24 bs=256 chunk_dur=5s
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/shov44gs/checkpoints/epoch=24-step=19900.ckpt' #golden-dew-25 bs=128 chunk_dur=10s

#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/zr304efn/checkpoints/epoch=23-step=17448.ckpt' #flowing-sound-26 bs=96 chunk_dur=15s

#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/8jtiowyz/checkpoints/epoch=46-step=38963.ckpt' #jumping-breeze-27 bs=64 chunk_dur=20s

#CHECKPOINT = '/speech/dbwork/mul/spielwiese3/demancum/cocola/e3xubuzt/checkpoints/epoch=38-step=245622.ckpt' #frosty-silence-39 musdb 32bs test
#CHECKPOINT = '/speech/dbwork/mul/spielwiese3/demancum/cocola_hpss/ciflwfwc/checkpoints/epoch=59-step=377880.ckpt' #astral-valley-21 more_negative_RAND_MASK_DOUBLE_CHANNEL
CHECKPOINT = '/speech/dbwork/mul/spielwiese3/demancum/cocola_hpss/10izn5cw/checkpoints/checkpoint-epoch=119-val_loss=0.00.ckpt' #young-sun-27 No_double_neg_HPSS


model = CoCola.load_from_checkpoint(CHECKPOINT)
trainer = Trainer()


/home/demancum/miniconda3/envs/cocola/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/demancum/miniconda3/envs/cocola/lib/python3.11 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## Setting embedding mode for the tests (for DOUBLE_CHANNEL_HARMONIC_PERCUSSIVE models only)

In [7]:
model.set_embedding_mode(constants.EmbeddingMode.BOTH) # constants.EmbeddingMode.PERCUSSIVE, constants.EmbeddingMode.BOTH or constants.EmbeddingMode.HARMONIC

## MOISESDB

In [10]:
dataset = MoisesdbContrastivePreprocessed(
    root_dir='/disk1/demancum/moisesdb_contrastive',
    preprocess=False
)

_, _, test_dataset = random_split(
    dataset=dataset, lengths=[0.8, 0.1, 0.1], generator=torch.Generator().manual_seed(42))

test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )


INFO:Found original dataset at /disk1/demancum/moisesdb_contrastive/moisesdb_v0.1.
INFO:Found preprocessed dataset at /disk1/demancum/moisesdb_contrastive/preprocessed_hpss.


In [9]:
trainer.test(model=model, dataloaders=test_dataloader)

You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


Testing DataLoader 0: 100%|██████████| 32/32 [00:02<00:00, 15.62it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │       0.5732421875        │
│         test_loss         │    1.7694923877716064     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.7694923877716064, 'test_accuracy': 0.5732421875}]

## slakh2100

In [42]:
test_dataset = Slakh2100ContrastivePreprocessed(
    root_dir='/disk1/demancum/slakh2100_contrastive',
    download=False,
    preprocess=False,
    split="test")
    

test_dataloader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )

trainer.test(model=model, dataloaders=test_dataloader)

INFO:Found original dataset split test at /disk1/demancum/slakh2100_contrastive/original/slakh2100_redux_16k/test.
INFO:Found preprocessed dataset split test at /disk1/demancum/slakh2100_contrastive/preprocessed_hpss/test.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]


Testing DataLoader 0: 100%|██████████| 248/248 [00:12<00:00, 19.86it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6280241935483871     │
│         test_loss         │    1.4796663522720337     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.4796663522720337, 'test_accuracy': 0.6280241935483871}]

## COCOCHORALES

In [45]:
test_dataset = CocoChoralesContrastivePreprocessed(
    root_dir='/disk1/demancum/coco_chorales_contrastive',
    download=False,
    preprocess=False,
    split="test",
    ensemble="*")
    
test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )


INFO:Found original dataset split test at /disk1/demancum/coco_chorales_contrastive/original/test.
INFO:Found preprocessed dataset split test at /disk1/demancum/coco_chorales_contrastive/preprocessed_hpss/test.


In [46]:
trainer.test(model=model, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]


Testing DataLoader 0: 100%|██████████| 9338/9338 [02:11<00:00, 70.93it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.997911758406511     │
│         test_loss         │   0.0055684843100607395   │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.0055684843100607395, 'test_accuracy': 0.997911758406511}]

## MusDB


In [11]:
from contrastive_model.contrastive_model_data import hpss

# Using train split since it is not used from training
test_dataset = MusdbContrastivePreprocessed(
    root_dir='/disk1/demancum/musdb_contrastive',
    download=False,
    preprocess=True, # Need to preprocess now because it is not used at training time
    split="train",
    chunk_duration=5,
    target_sample_rate=16000,
    generate_submixtures=True,
    device="cuda",
    preprocess_transform=hpss
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )

trainer.test(model=model, dataloaders=test_dataloader)



INFO:Found original dataset split train at /disk1/demancum/musdb_contrastive/original/train.
INFO:Found preprocessed dataset split train at /disk1/demancum/musdb_contrastive/preprocessed_hpss/train.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


Testing DataLoader 0: 100%|██████████| 4619/4619 [01:04<00:00, 71.74it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9387313271270837     │
│         test_loss         │    0.16714246571063995    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.16714246571063995, 'test_accuracy': 0.9387313271270837}]